In [51]:
%matplotlib inline

import numpy as np
import nltk as nltk #nlp module
import matplotlib as plot #visuals
import sklearn as skl
import pandas as pd
import spacy #nlp module
import re
import random
from collections import defaultdict, Counter
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import Ridge
from sklearn.cross_validation import train_test_split

nlp = spacy.load('en')
from spacy.en import English
parser = English()

In [44]:
textRead = pd.ExcelFile('factCheck.xlsx')
textRead = textRead.parse('Sheet1')
textRead.columns = ['Json', 'Accuracy', 'Title', 'Genre', 'KeywordName', 'Occupation', 'Location', 'PoliticalParty', 
                    '1', '2', '3', '4', '5', 'Source', 'Url']
textRead.head()

,Json,Accuracy,Title,Genre,KeywordName,Occupation,Location,PoliticalParty,1,2,3,4,5,Source,Url
0,10540.json,half-true,When did the decline of coal start? It started...,"energy,history,job-accomplishments",scott-surovell,State delegate,Virginia,democrat,0,0,1,1,0,a floor speech.,"<p>Del. Scott Surovell&rsquo;s floor <a href=""..."
1,324.json,mostly-true,"Hillary Clinton agrees with John McCain ""by vo...",foreign-policy,barack-obama,President,Illinois,democrat,70,71,160,163,9,Denver,"Barack Obama campaign Web site, <a href=""http:..."
2,1123.json,False,Health care reform legislation is likely to ma...,health-care,blog-posting,NaN,NaN,none,7,19,3,5,44,a news release,"<p> News release attributed to Matt Barber, &l..."
3,9028.json,half-true,The economic turnaround started at the end of ...,"economy,jobs",charlie-crist,NaN,Florida,democrat,15,9,20,19,2,an interview on CNN,"<p>CNN transcript, <a href=""http://transcripts..."
4,12465.json,True,The Chicago Bears have had more starting quart...,education,robin-vos,Wisconsin Assembly speaker,Wisconsin,republican,0,3,2,5,1,a an online opinion-piece,"<p>Right Wisconsin, <a href=""http://www.rightw..."


In [46]:
sentenceFrame = textRead[textRead.columns[2]]

Start cleaning up the data. Start to create a bag of words matrix to give a numerical representation of which words are in which phrase that we are predicting on. Algorithms predict on numerical attributes, not categorical. 

First, find the unique words across the entire column. Set up a matrix where the row is the phrase and each respective column is one unique word. Fill the cell with the number of times that each word in that headline. 

Obviously most columns will have a zero, unless there is a particularly common word among the phrases. 

In [47]:
uniqueWords = list(set(" ".join(sentenceFrame).split(" ")))
def make_matrix(phrase, vocab):
    matrix = []
    for phrase in sentenceFrame:
        # Count each word in the headline, and make a dictionary.
        counter = Counter(phrase)
        # Turn the dictionary into a matrix row using the vocab.
        row = [counter.get(w, 0) for w in vocab]
        matrix.append(row)
    df = pd.DataFrame(matrix)
    df.columns = uniqueWords
    return df

print(make_matrix(sentenceFrame, uniqueWords))

        401(k)s.  chatter  minority  Benghazi  88  taxpayers  murders  test.  \
0    0         0        0         0         0   0          0        0      0   
1    0         0        0         0         0   0          0        0      0   
2    0         0        0         0         0   0          0        0      0   
3    0         0        0         0         0   0          0        0      0   
4    0         0        0         0         0   0          0        0      0   
5    0         0        0         0         0   0          0        0      0   
6    0         0        0         0         0   0          0        0      0   
7    0         0        0         0         0   0          0        0      0   
8    0         0        0         0         0   0          0        0      0   
9    0         0        0         0         0   0          0        0      0   
10   0         0        0         0         0   0          0        0      0   
11   0         0        0         0     

Remove all the punctuation from the sentences and make every letter lowercase. 

In [31]:
newPhrase = [re.sub(r'[^\w\s\d]','',h.lower()) for h in sentenceFrame]
# Replace sequences of whitespace with a space character.
newPhrase = [re.sub("\s+", " ", h) for h in newPhrase]

uniqueWords = list(set(" ".join(newPhrase).split(" ")))
# We've reduced the number of columns in the matrix a bit.
print(make_matrix(newPhrase, uniqueWords))

        chatter  minority  88  taxpayers  murders  history  wisconsinites  \
0    0        0         0   0          0        0        0              0   
1    0        0         0   0          0        0        0              0   
2    0        0         0   0          0        0        0              0   
3    0        0         0   0          0        0        0              0   
4    0        0         0   0          0        0        0              0   
5    0        0         0   0          0        0        0              0   
6    0        0         0   0          0        0        0              0   
7    0        0         0   0          0        0        0              0   
8    0        0         0   0          0        0        0              0   
9    0        0         0   0          0        0        0              0   
10   0        0         0   0          0        0        0              0   
11   0        0         0   0          0        0        0              0   

Remove unnecessary stopwords. 

In [32]:
# Read in and split the stopwords file.
with open("stop_words.txt", 'r') as f:
    stopwords = f.read().split("\n")

# Do the same punctuation replacement that we did for the headlines, 
# so we're comparing the right things.
stopwords = [re.sub(r'[^\w\s\d]','',s.lower()) for s in stopwords]

uniqueWords = list(set(" ".join(newPhrase).split(" ")))
# Remove stopwords from the vocabulary.
uniqueWords = [w for w in uniqueWords if w not in stopwords]

# We're down to 34 columns, which is way better!
print(make_matrix(newPhrase, uniqueWords))

#Down to 960 columns! 

     chatter  minority  88  taxpayers  murders  history  wisconsinites  \
0          0         0   0          0        0        0              0   
1          0         0   0          0        0        0              0   
2          0         0   0          0        0        0              0   
3          0         0   0          0        0        0              0   
4          0         0   0          0        0        0              0   
5          0         0   0          0        0        0              0   
6          0         0   0          0        0        0              0   
7          0         0   0          0        0        0              0   
8          0         0   0          0        0        0              0   
9          0         0   0          0        0        0              0   
10         0         0   0          0        0        0              0   
11         0         0   0          0        0        0              0   
12         0         0   0          0 

Uh this is where I start to get a little stuck. 

We have to start training the data but a lot of this is probably wrong. Hasn't been tested yet.  

In [52]:
train_rows = 768
# Set a seed to get the same "random" shuffle every time.
random.seed(1)

# Shuffle the indices for the matrix.
indices = list(range(features.shape[0]))
random.shuffle(indices)

# Create train and test sets.
train = features[indices[:train_rows], :]
test = features[indices[train_rows:], :]
train_upvotes = submissions["upvotes"].iloc[indices[:train_rows]]
test_upvotes = submissions["upvotes"].iloc[indices[train_rows:]]
train = numpy.nan_to_num(train)

# Run the regression and generate predictions for the test set.
reg = Ridge(alpha=.1)
reg.fit(train, train_upvotes)
predictions = reg.predict(test)


NameError: name 'features' is not defined